In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from positionCorrection import positionCorrection as correct
from mpl_toolkits.mplot3d import Axes3D
import math

In [2]:
snapData = np.genfromtxt('data/data_spacing.txt')
snapshots = snapData[:,0]
scale = snapData[:,1]
redshift = snapData[:,2]


In [3]:
# reading in the data for all subhalos in identified groups
snaps = np.array([135, 131, 126, 117, 103, 93, 84, 75])
reds = np.array([redshift[snapshots == i][0] for i in snaps])
scales = 1/(1+reds)

primFiles = ['data/data_primaries_LMCAnalogs_'+str(i)+'.csv' for i in snaps]
secoFiles = ['data/data_secondaries_LMCAnalogs_'+str(i)+'.csv' for i in snaps]


In [4]:
#######################################
# Mass cuts for LMC/SMC analogs       #
#######################################
numberMassAnalogs = []

primaryCols = ['Group Number', 'Subhalo ID','Redshift', 'Mass at z=0', 'Max Mass',
           'Max Mass Snap', 'Redshift at Max Mass', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
           'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass']
secondaryCols = ['Group Number', 'Subhalo ID','Redshift', 'Mass at z=0', 'Max Mass',
           'Max Mass Snap', 'Redshift at Max Mass', 'Rvir at z=0', 'Rvir at Max Mass', 'Pos x',
           'Pos y', 'Pos z', 'Vel x', 'Vel y', 'Vel z', 'AM Stellar Mass','Escape Speed z=0','Escape Speed Max Mass']

for i in range(len(snaps)):
    #####################
    # mass analog cuts! #
    #####################
    prims = pd.read_csv(primFiles[i])
    secos = pd.read_csv(secoFiles[i])

    # defining the mass ratio according to stellar mass
    massRatio = secos['AM Stellar Mass']/prims['AM Stellar Mass'] 
    
    #SMC/LMC stellar mass ratio between 1/15 and 1/5
    SMCratioCut = np.logical_and(massRatio > 1/15, massRatio < 1/5)
    
    #SMC stellar mass > 2e8
    SMCmassCut = (0.02 <= secos['AM Stellar Mass'])
    
    #creating mass mask for pairs that pass the above 3 criteria
    massCuts = SMCratioCut&SMCmassCut
    
    #creating mask for pairs that have LMCs, but do not have an LMC-like companion
    cutNoSMCs = ~SMCratioCut&~SMCmassCut
    
    #creating data tables of mass cuts  (bound and unbound)
    prims_massCut = prims[massCuts]
    secos_massCut = secos[massCuts]
    numberMassAnalogs.append(len(prims_massCut))
    
    LMCFileToSave = 'data/data_primaries_massAnalogs_'+str(snaps[i])+'.csv'
    SMCFileToSave = 'data/data_secondaries_massAnalogs_'+str(snaps[i])+'.csv'
    
    primsData = pd.DataFrame(data = prims_massCut, columns=primaryCols)
    primsData.to_csv(LMCFileToSave,index=False,header=True)
    
    secosData = pd.DataFrame(data = secos_massCut, columns=secondaryCols)
    secosData.to_csv(SMCFileToSave,index=False,header=True)
    